## Курсова машін льорнінінг

### Data loading

#### imports

In [7]:
import pandas as pd
import re
import os
import pandasql as ps

#### functions

In [8]:
import csv


def standardize_columns(df, column_mapping):
    """
    Standardize column names based on a mapping dictionary.
    :param df: DataFrame with raw data.
    :param column_mapping: Dictionary where keys are possible column names and values are standardized names.
    :return: DataFrame with standardized column names.
    """
    standardized_columns = {}
    for col in df.columns:
        # Map column names using the dictionary, or keep the original name if not found
        standardized_columns[col] = column_mapping.get(col, col)
    
    df.rename(columns=standardized_columns, inplace=True)
    return df

def detect_csv_format(file_path):
    """
    Automatically detect the format of a CSV file (delimiter, quotechar, etc.).
    :param file_path: Path to the CSV file.
    :return: Dictionary with parameters for reading the file.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            sample = f.read(1024)  # Read the first 1024 characters for analysis
            sniffer = csv.Sniffer()
            dialect = sniffer.sniff(sample)
            return {
                'delimiter': dialect.delimiter,
                'quotechar': dialect.quotechar
            }
    except Exception as e:
        print(f"Failed to detect the format of the file {file_path}: {e}")
        # Return fallback delimiters in order of likelihood
        return {'delimiter': ';', 'quotechar': '"'}


def process_salary_data(input_folder, output_file):
    """
    Processes all salary files, standardizes columns, adds the period (half-year), and combines them into one dataset.
    :param input_folder: Path to the folder with input files.
    :param output_file: Path to the output CSV file.
    """
    combined_data = []  # List to store processed data

    # Column mapping for standardization
    column_mapping = {
        "Зарплата / дохід в ІТ у $$$ за місяць, лише ставка після сплати податків": "salary",
        "Зарплата": "salary",
        "Salary": "salary",
        "salary": "salary",
        "Зарплата в месяц": "salary",
        "Зарплата за місяць, net, після податків": "salary",
        "Зарплата у $$$ за місяць, лише ставка після сплати податків": "salary",
        "Чи отримуєте ви грошові бонуси до зарплати?": "bonuses flag",
        "Вкажіть суму цього бонуса у $$$ після податків": "bonuses amount",
        "Зарплата.в.месяц": "salary",
        "Вкажіть суму цього бонуса у  після податків": "bonuses amount",
        "Зарплата / дохід у $$$ за місяць, лише ставка ЧИСТИМИ - після сплати податків": "salary",
        "Зарплата / дохід в IT у $$$ за місяць": "salary",
        """Зарплата / дохід в ІТ у $$$ за місяць, лише ставка 
ЧИСТИМИ - після сплати податків""": "salary",
        "Зарплата / дохід у $$$ за місяць, лише ставка після сплати податків" : "salary",
        """Зарплата / дохід у $$$ за місяць, лише ставка 
ЧИСТИМИ - після сплати податків""": "salary",
    """\"Зарплата / дохід у $$$ за місяць, лише ставка 
ЧИСТИМИ - після сплати податків",Оберіть вашу основну посаду""": "salary",
        """Зарплата / дохід в IT у $$$ за місяць, лише ставка 
ЧИСТИМИ - після сплати податків""": "salary",
        



        
        "Должность": "position",
        "Оберіть вашу основну посаду": "position",
        "Оберіть вашу посаду": "position",
        "Посада": "position",
        "Position": "position",
        "position": "position",
        "Спеціалізація": "position",
        
        
        "Ваш тайтл на цій посаді": "seniority",
        "Оберіть ваш тайтл або роль у компанії": "seniority",
        "Общий опыт работы": "general experience seniority",
        "Опыт работы на текущем месте": "local experience seniority",
        "Загальний стаж роботи за спеціальністю": "general experience seniority",
        "Стаж на поточному місці роботи": "local experience seniority",
        "Ваш тайтл": "seniority",
        "exp": "general experience seniority",
        "current_job_exp": "local experience seniority",
        "experience": "general experience seniority",
        
        
        "freq": "frequency"
    }

    # Read all files in the input folder
    input_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]
    input_files = sorted(input_files, key=lambda x: int(re.search(r'\d+', x).group()))  # Sort by year

    for input_file in input_files:
        # Determine year and month from the file name
        match = re.search(r'(\d{4})_(\w+)_.*\.csv', input_file)
        if not match:
            print(f"Skipping file {input_file}: Could not determine year and month.")
            continue
        
        year, month = match.groups()
        year = int(year)
        
        # Determine half-year period
        if month.lower() in ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'june']:
            half_year = f"{year}_H1"  # First half-year
        else:
            half_year = f"{year}_H2"  # Second half-year

        # Read the file with automatic format detection
        file_path = os.path.join(input_folder, input_file)
        try:
            if (half_year == "2021_H2" or half_year == "2022_H2" or half_year == "2023_H1"):
                csv_format ={'delimiter': ';', 'quotechar': '"'}
            else: 
                csv_format = detect_csv_format(file_path)  # Detect the format of the file
                
            df = pd.read_csv(file_path, on_bad_lines='skip', **csv_format)
            if (half_year == "2021_H1"):
                df.rename(columns={"Загальний стаж роботи за спеціальністю": "general_experience"}, inplace=True)
                
            if (half_year == "2022_H1"):
                # Видалення записів, де "city" = "Всі міста" або "experience" = "Весь досвід"
                df = df[~((df["city"] == "Всі міста") | (df["experience"] == "Весь досвід"))]

            # Remove empty rows
            df.dropna(how='all', inplace=True)

            # Standardize column names
            df = standardize_columns(df, column_mapping)

            # Add the half-year period column
            df['period'] = half_year
            print(f"half year: {half_year} csv format: {csv_format}")
            # Append the processed DataFrame to the combined list
            combined_data.append(df)
        except Exception as e:
            print(f"Error reading file {input_file}: {e}")
            continue

    # Combine all DataFrames into one
    combined_df = pd.concat(combined_data, ignore_index=True)

    # Select only the required columns
    final_columns = ['period', 'position', "seniority", "general experience seniority",
                     "local experience seniority", 'salary', "bonuses flag", "bonuses amount", "frequency"]
    combined_df = combined_df[final_columns]

    # Save the combined dataset to a file
    combined_df.to_csv(output_file, index=False)
    print(f"Final dataset saved to: {output_file}")
def get_next_filename(base_filename, folder):
    """
    Генерує унікальну назву файлу, додаючи +1 до номера.
    """
    if not os.path.exists(folder):
        os.makedirs(folder)

    files = os.listdir(folder)

    matching_files = [f for f in files if f.startswith(base_filename) and f.endswith(".csv")]

    max_number = 0
    for file in matching_files:
        try:
            number = int(file.replace(base_filename, "").replace(".csv", "").strip("_"))
            if number > max_number:
                max_number = number
        except ValueError:
            continue

    next_number = max_number + 1
    return os.path.join(folder, f"{base_filename}_{next_number}.csv")


#### variables 

In [9]:
input_folder = "unprocessed_salary"
output_folder = "raw_salary_compile" 

#### main script

In [10]:
output_path = get_next_filename("combined_salary", output_folder)

print(f"Обробляємо файли: {input_folder}\\")
process_salary_data(input_folder, output_path)
print(f"Результат збережено в: {output_path}")

Обробляємо файли: unprocessed_salary\
half year: 2015_H2 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2015_H1 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2016_H2 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2016_H1 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2017_H2 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2017_H1 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2018_H2 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2018_H1 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2019_H2 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2019_H1 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2020_H2 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2020_H1 csv format: {'delimiter': ',', 'quotechar': '"'}
half year: 2021_H2 csv format: {'delimiter': ';', 'quotechar': '"'}
half year: 2021_H1 csv format: {'delimiter': ',', 'quotechar': '"'}
half year:

In [11]:


data = pd.read_csv(output_path, low_memory=False)
query = """
    SELECT count(*)
    FROM data
    where salary is NULL
    limit 100
"""

# Виконання запиту
unique_bonuses_flag = ps.sqldf(query, locals())

# Виведення результату
print(unique_bonuses_flag)

   count(*)
0       116


In [12]:

query = """
    SELECT distinct 
    FROM data
    where salary is NULL
    limit 100
"""

# Виконання запиту
unique_bonuses_flag = ps.sqldf(query, locals())

# Виведення результату
print(unique_bonuses_flag)

FileNotFoundError: [Errno 2] No such file or directory: 'raw_salary_compile/combined_salary_2.csv'